# Meta update learning attempts to use transfer learning

It's a significant refactor and review to go over this code and remember what exactly everything is doing, but it would be nice to get this running for ACMBC at least

Need to regenerate a dataset that fits the hyperparameters of the original paper here.
https://arxiv.org/abs/2003.06113


## Imports

In [2]:
import sys, os
sys.path.append(os.path.realpath(".."))
os.environ["TF_XLA_FLAGS"]="--tf_xla_cpu_global_jit"
os.environ["EEG_ROOT"] = "/home/mohammed/eeg-tasks"

import tensorflow as tf
# tf.enable_eager_execution()
# tf.enable_v2_behavior()
# sess = tf.InteractiveSession()
# sess.as_default()

import util_funcs
from importlib import reload
reload(util_funcs)
import data_reader as read
import pandas as pd
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
import constants
import clinical_text_analysis as cta
import tsfresh
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, roc_curve
from os import path
# import predictSeizureConvExp as psce
import keras_models.dataGen as dg
from addict import Dict
# reload(psce)
import ensembleReader as er
reload(er)
import time
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, log_loss
from functools import lru_cache

In [15]:
eds = er.EdfDatasetSegments(pre_cooldown=0, post_cooldown=0, sample_time=0, num_seconds=400/250.0, n_process=20)
train_label_files_segs = eds.get_train_split()
valid_label_files_segs = eds.get_valid_split()
test_label_files_segs = eds.get_test_split()

Starting 20 processes
retrieving: 0
Starting 20 processes
retrieving: 0
retrieving: 1000
retrieving: 2000
retrieving: 3000
starting custom_annotater
Starting 20 processes
retrieving: 0
retrieving: 1000
retrieving: 2000
starting custom_annotater
Starting 20 processes
retrieving: 0
starting custom_annotater
Starting 20 processes
retrieving: 0


In [16]:
train_edss = er.EdfDatasetSegmentedSampler(segment_file_tuples=train_label_files_segs, include_seizure_type=True, include_segment=True, n_process=int(12), gap=pd.Timedelta(seconds=400/250.0), shuffle=False)
valid_edss = er.EdfDatasetSegmentedSampler(segment_file_tuples=valid_label_files_segs, include_seizure_type=True, include_segment=True, n_process=int(12), gap=pd.Timedelta(seconds=400/250.0),  shuffle=False)
test_edss = er.EdfDatasetSegmentedSampler(segment_file_tuples=test_label_files_segs, include_seizure_type=True, include_segment=True, n_process=int(12), gap=pd.Timedelta(seconds=400/250.0),  shuffle=False)

In [17]:
len(train_edss), len(valid_edss), len(test_edss)

(958217, 189031, 331980)

In [6]:
import pickle as pkl

In [18]:
train_edss.n_process = 24
valid_edss.n_process = 24
test_edss.n_process = 24

In [19]:
len(train_edss)

958217

In [20]:
pkl.dump(train_edss.sampleInfo.to_dict(), open("/datadrive/TUH_EEG/mups/train_index.pkl", "wb"))
pkl.dump(valid_edss.sampleInfo.to_dict(), open("/datadrive/TUH_EEG/mups/valid_index.pkl", "wb"))
pkl.dump(test_edss.sampleInfo.to_dict(), open("/datadrive/TUH_EEG/mups/test_index.pkl", "wb"))

In [ ]:
import math
for i in range(math.ceil(len(train_edss)/100000)):
    train_edss_subsample = train_edss[i*100000:min((i+1)*100000, len(train_edss))]
    pkl.dump(train_edss_subsample, open(f"/datadrive/TUH_EEG/mups/train_mups_{i}.pkl","wb"))
    del train_edss_subsample

Starting 24 processes
retrieving: 0
retrieving: 250
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
retrieving: 500
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 4, less than 7 requested!!!
read 4, less than 7 requested!!!
read 4, less than 7 requested!!!
retrieving: 750
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
retrieving: 1000
read 4, less than 7 requested!!!
read 4, less than 7 requested!!!
read 4, less than 7 requested!!!
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
retrieving: 1250
read 6, less than 7 requested!!!
read 5, less than 7 requested!!!
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 5, less than 7 requested!!!
read 5, less than 7 requ

read 4, less than 7 requested!!!
retrieving: 13000
retrieving: 13250
retrieving: 13500
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
read 6, less than 7 requested!!!
retrieving: 13750
retrieving: 14000
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
retrieving: 14250
read 6, less than 7 requested!!!
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
retrieving: 14500
retrieving: 14750
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 5, less than 7 requested!!!
read 6, less than 7 requested!!!
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
retrieving: 15000
read 6, less 

read 6, less than 7 requested!!!
retrieving: 29500
retrieving: 29750
read 6, less than 7 requested!!!
read 5, less than 7 requested!!!
read 6, less than 7 requested!!!
read 5, less than 7 requested!!!
read 6, less than 7 requested!!!
read 5, less than 7 requested!!!
retrieving: 30000
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
retrieving: 30250
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 5, less than 7 requested!!!
retrieving: 30500
read 5, less than 7 requested!!!
read 6, less than 7 requested!!!
read 5, less than 7 requested!!!
retrieving: 30750
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 5, less than 7 requested!!!
read 6, less than 7 requested!!!
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
retrieving: 31000
retrieving: 31250
read 6, less than 7 requested!!

read 4, less than 7 requested!!!
read 6, less than 7 requested!!!
retrieving: 44750
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
retrieving: 45000
read 4, less than 7 requested!!!
read 6, less than 7 requested!!!
read 4, less than 7 requested!!!
read 4, less than 7 requested!!!
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
retrieving: 45250
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 4, less than 7 requested!!!
read 4, less than 7 requested!!!
read 4, less than 7 requested!!!
read 6, less than 7 requested!!!
read 4, less than 7 requested!!!
read 6, less than 7 requested!!!
read 4, less than 7 requested!!!
read 4, less than 7 requested!!!
read 6, less than 7 requested!!!
retrieving: 45500
retrieving: 45750
retrieving: 46000
retrieving: 46250
retrieving: 4650

read 5, less than 7 requested!!!
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
retrieving: 62500
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
retrieving: 62750
read 6, less than 7 requested!!!
read 4, less than 7 requested!!!
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 4, less than 7 requested!!!
read 4, less than 7 requested!!!
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 4, less than 7 requested!!!
read 6, less than 7 requested!!!
read 4, less than 7 requested!!!
read 4, less than 7 requested!!!
retrieving: 63000
retrieving: 63250
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
retrieving: 63500
retrieving: 63750
read 5, less than 7 requested!!!
r

...

read 6, less than 7 requested!!!
read 4, less than 7 requested!!!
read 4, less than 7 requested!!!
read 5, less than 7 requested!!!
read 4, less than 7 requested!!!
read 5, less than 7 requested!!!
read 4, less than 7 requested!!!
read 5, less than 7 requested!!!
retrieving: 360000
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
retrieving: 360250
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
read 4, less than 7 requested!!!
read 4, less than 7 requested!!!
read 4, less than 7 requested!!!
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
read 4, less than 7 requested!!!
read 4, less than 7 requested!!!
read 4, less than 7 requested!!!
retrieving: 360500
read 5, less than 7 requested!!!
read 4, less than 7 requested!!!
read 5, less than 7 requested!!!
read 4, less than 7 requested!!!
read 5, less than 7 requested!!!
read 4, less than 7

read 5, less than 7 requested!!!
read 4, less than 7 requested!!!
read 5, less than 7 requested!!!
read 4, less than 7 requested!!!
read 4, less than 7 requested!!!
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 4, less than 7 requested!!!
read 4, less than 7 requested!!!
read 4, less than 7 requested!!!
read 4, less than 7 requested!!!
read 6, less than 7 requested!!!
read 4, less than 7 requested!!!
read 4, less than 7 requested!!!
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
retrieving: 366500
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
read 4, less than 7 requested!!!
read 5, less than 7 requ

read 4, less than 7 requested!!!
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 4, less than 7 requested!!!
read 4, less than 7 requested!!!
retrieving: 378000
retrieving: 378250
read 5, less than 7 requested!!!
read 6, less than 7 requested!!!
read 5, less than 7 requested!!!
read 6, less than 7 requested!!!
read 5, less than 7 requested!!!
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 4, less than 7 requested!!!
read 6, less than 7 requested!!!
read 4, less than 7 requested!!!
read 6, less than 7 requested!!!
read 4, less than 7 requested!!!
retrieving: 378500
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 4, less than 7 requested!!!
read 4, less than 7 requested!!!
read 4, less than 7 requested!!!
retrieving: 378750
read 6, less than 7 requested!!!
read 4, less than 7 requested!!!


retrieving: 395250
retrieving: 395500
retrieving: 395750
retrieving: 396000
read 6, less than 7 requested!!!
read 4, less than 7 requested!!!
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 4, less than 7 requested!!!
read 4, less than 7 requested!!!
retrieving: 396250
retrieving: 396500
retrieving: 396750
retrieving: 397000
retrieving: 397250
retrieving: 397500
retrieving: 397750
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
retrieving: 398000
retrieving: 398250
retrieving: 398500
retrieving: 398750
read 5, less than 7 requested!!!
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 5, less than 7 requested!!!
read 6, less than 7 requested!!!
read 5, less than 7 requested!!!
retrieving: 399000
retrieving: 399250
retrieving: 399500
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 5, less than 7 requested!!!
read 5, less than 7 requested!!

read 5, less than 7 requested!!!
retrieving: 413500
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 4, less than 7 requested!!!
read 4, less than 7 requested!!!
read 4, less than 7 requested!!!
retrieving: 413750
retrieving: 414000
read 6, less than 7 requested!!!
read 5, less than 7 requested!!!
read 6, less than 7 requested!!!
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
read 6, less than 7 requested!!!
retrieving: 414250
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 4, less than 7 requested!!!
read 4, less than 7 requested!!!
read 4, less than 7 requested!!!
retrieving: 414500
read 6, less than 7 requested!!!
read 6, less than 7 requested!!!
read 5, less than 7 requested!!!
read 6, less than 7 requested!!!
read 5, less than 7 requested!!!
read 5, less than 7 requested!!!
retrieving: 414750
read 5, less than 7 requested!!!
read 5, less than 7 requeste

In [ ]:
for i in range(math.ceil(len(valid_edss)/100000)):
    valid_edss_subsample = valid_edss[i*100000:min((i+1)*100000, len(valid_edss))]
    pkl.dump(valid_edss_subsample, open(f"/datadrive/TUH_EEG/mups/valid_mups_{i}.pkl","wb"))
    del valid_edss_subsample

In [ ]:
for i in range(math.ceil(len(test_edss)/100000)):
    test_edss_subsample = test_edss[i*100000:min((i+1)*100000, len(test_edss))]
    pkl.dump(test_edss_subsample, open(f"/datadrive/TUH_EEG/mups/test_mups_{i}.pkl","wb"))
    del test_edss_subsample

In [25]:
test_edss[0]

(array([[  0.20713515,  -0.29970871,   0.29675997, ...,  -0.0524678 ,
           0.34311764,  -1.30412493],
        [  1.44053733,  -2.04108009,   2.12565027, ...,  -0.28453806,
           2.40169056,  -9.01400779],
        [  4.45403603,  -6.13796042,   6.82664799, ...,  -0.56251201,
           7.65735767, -27.73326166],
        ...,
        [  2.08322786,  -3.34398133,   5.13508745, ...,   0.95236003,
          -5.02606334,   2.42356812],
        [  2.09585184,  -2.6209878 ,   6.55974814, ...,   2.08096889,
          -2.76738134,   3.03221213],
        [  2.7145432 ,  -2.75183502,   7.71071189, ...,   2.15033429,
          -0.28507881,   1.75625638]]),
 (False,
  'bckg',
  '/datadrive/TUH_EEG/edf/dev/01_tcp_ar/050/00005031/s003_2011_05_19/00005031_s003_t001.edf'))

In [27]:
from torch.utils.data import DataLoader

In [31]:
class TUHDataLoader(DataLoader):
    def __init__(self):
        parent.__init__

In [ ]:
train_edss.data

In [ ]:
%%bash
scancel 9781732

In [ ]:
plt.plot(train_edss[1][0])

In [ ]:
len(train_edss)

In [ ]:
train_edss[0][0].shape